С помощью теоретических материалов, представленных в книгах Спонга и статье Slotine and Li был реализован адаптивный контроллер, направленный на нивелирование всех эффектов неопределенностей в параметрах робота. Управление происходит 6-DOF манипулятором UR5.

Основные уравнения, используемые при реализации контроллера:

Управляющий закон для робастного управления:
$$ u = \hat{M}(q)a + \hat{C}(q,\dot{q})v + \hat{g}(q) - Kr $$

Где механические параметры могут быть параметризованы как: 
$$ Y(q, \dot{q}, a, v)\hat{\theta} =  \hat{M}(q)a + \hat{C}(q,\dot{q})v + \hat{g}(q)$$

Получаем новый закон управления:
$$ u = Y(q, \dot{q}, a, v)\hat{\theta} - Kr $$

Где $ r = \dot{\tilde{q}} + \Lambda \tilde{q}$

Таким образом мы получили управляющий закон, использующий ```jointTorqueRegressor``` $Y$ и оценку параметров $\hat{\theta}$

Для того, чтобы управление адаптировалось к манипулятору и нагрузке, необходим закон адаптации, который на каждом шаге будет изменять параметры в сторону уменьшения ошибки.

Для этой задачи используется закон адаптации, основанный на принципах градиентного спуска:
$$ \dot{\hat{\theta}} = -{\Gamma}^{-1}Y^{T}r $$

Диагональная матрица $\Gamma$ размера $Y_{rows}{\times}Y_{rows}$ здесь - своего рода коэффициент, определяющий скорость адаптации (чем выше значения, тем медленнее и аккуратнее происходит адаптация) \
Диагональная матрица $\Lambda$ размера $q_{rows}{\times}q_{rows}$ это тоже коэффициенты, которые отвечают за пропорциональную составляющую ошибки.

Параметры ${\theta}$ это динамические параметры звеньев, включающие в себя массу, элементы матрицы инерции и нелинейности. Для получения общей матрицы всех звеньев строки параметров конкатенируются, в данном случае вектор параметров $\theta$ имеет размерность $1{\times}60$

Непосредственно имплементация управления начинается с создания экземпляров среды симуляции с моделью робота в Mujoco. \
Всего было создано три экземпляра: два для inverse dynamics контроллера с известными параметрами робота и с неизвестной массой эндэффектора и один для адаптивного контроллера с неизвестной массой эндэффектора и шумом во всех других параметрах.

Внутри симулятора происходит инициализация модели Mujoco, если необходимо, в модели изменяется масса эндэффектора. По большому счету, внутри симуляции модели не отличаются друг от друга (только добавлением массы)

После этого происходит инициализация модели pinocchio из того же файла, что и Mujoco-модель, но в ее параметры $\theta$, не равные нулю, добавляется шум до 20 процентов от значения.

Резюмируя: 
- для обычной обратной динамики:
    - в симуляции оригинальная модель
    - контроллер использует данные оригинальной модели
- для обратной динамики с добавленным весом:
    - в симуляторе используется модель с добавленным весом
    - контроллер использует данные оригинальной модели
- для адаптивного контроллера с добавленным весом:
    - в симуляторе используется модель с добавленным весом
    - контроллер использует данные оригинальной модели с добавленным шумом


Ниже представлены результаты тестирования, значение добавленной массы равно килограмму

##### Сходимость координат манипулятора к точке для inverse dynamics и ошибка координат
<img src="logs/plots/point_inv_din.png" alt="Логотип" width="700" height="500" /> 
<img src="logs/plots/point_inv_din_errors.png" alt="Логотип" width="700" height="500" />

##### Сходимость координат манипулятора к точке для inverse dynamics с неизвестной массой и ошибка координат
<img src="logs/plots/point_inv_din_added_mass.png" alt="Логотип" width="700" height="500" />
<img src="logs/plots/point_inv_din_added_mass_errors.png" alt="Логотип" width="700" height="500" />

##### Сходимость координат манипулятора к точке для адаптивного контроллера с неизвестной массой и параметрами и ошибка координат
<img src="logs/plots/point_adaptive_added_mass.png" alt="Логотип" width="700" height="500" />
<img src="logs/plots/point_adaptive_added_mass_errors.png" alt="Логотип" width="700" height="500" />

##### Сходимость координат манипулятора к траектории для inverse dynamics и ошибка координат
<img src="logs/plots/inv_din.png" alt="Логотип" width="700" height="500" /> 
<img src="logs/plots/inv_din_errors.png" alt="Логотип" width="700" height="500" />

##### Сходимость координат манипулятора к траектории для inverse dynamics с неизвестной массой и ошибка координат
<img src="logs/plots/inv_din_added_mass.png" alt="Логотип" width="700" height="500" />
<img src="logs/plots/inv_din_added_mass_errors.png" alt="Логотип" width="700" height="500" />

##### Сходимость координат манипулятора к траектории для адаптивного контроллера с неизвестной массой и параметрами и ошибка координат
<img src="logs/plots/adaptive_added_mass.png" alt="Логотип" width="700" height="500" />
<img src="logs/plots/adaptive_added_mass_errors.png" alt="Логотип" width="700" height="500" />

На графиках можно заметить, что, в то время как inverse dynamics контроллер хорошо справляется с задачей при известном манипуляторе, он полностью проваливает задание, если масса эндэффектора неизвестна.

Адаптивный контроллер, в свою очередь, даже с учетом бОльшей неопределенности в виду шума в остальных параметрах, выполняет задание на том же уровне, что и inv dyn контроллер с детерминированным манипулятором.

В случае с адаптивным контроллером с неопределенностью и обычным без нее можно увидеть колебания ориентации эндэффектора, я полагаю, что они вызваны характером движения (все шарниры двигались по синусоидам), а для адаптивного еще и неопределенностями/величиной неизвестной массы. 

В любом случае, данные колебания могут быть уменьшены за счет более точного выбора коэффициентов.

К сожалению, мне не удалось добиться включения демпинга шарниров в систему в виду отсутствия времени на это (требуется подготовка к другому экзамену по семестровой дисциплине).

Ниже представлены видео движения манипуляторов.


##### Детерминированная обратная динамика, траектория и точка
<video controls src="logs/videos/inv_din.mp4" title="Title" width='700' height='500'></video> <video controls src="logs/videos/point_inv_din.mp4" title="Title" width='700' height='500'></video>

##### Обратная динамика с неопределенностью, траектория и точка
<video controls src="logs/videos/inv_din_add_mass.mp4" title="Title" width='700' height='500'></video> <video controls src="logs/videos/point_inv_din_add_mass.mp4" title="Title" width='700' height='500'></video>

##### Адаптированный контроллер с неопределенностями, траектория и точка
<video controls src="logs/videos/adaptive_add_mass_noise.mp4" title="Title" width='700' height='500'></video> <video controls src="logs/videos/point_adaptive_add_mass_noise.mp4" title="Title" width='700' height='500'></video>